In [ ]:
import sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.multi_sim import MultiSim
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.gps_importer import *

In [ ]:
tracks = []
v_average = 15
mu_corr = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
total_rsd = 900

In [ ]:
# here we make a function that generates a car with a given roll stiffness distribution and chassis roll stiffness
# the first and second arguments take the values of the first and second sweep variables that you give to run_sim
def gen_car_lltd(rsd, k_c):
    total_roll_stiffness = total_rsd * FTLB_TO_NM
    car = Car()
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.k_c = k_c
    car.description = f"rsd:{rsd:.2f} k_c:{k_c:.2f}"
    return car
# do a sweep of roll stiffness distribution and chassis roll stiffness
rsd_range = np.linspace(0.2, 0.8, 50)
# k_c_r = np.linspace(100, 10000, 50)
k_c_r = np.array([600, 1000, 1200, 1300, 1400, 1500, 1800, 2500, 999999999])#np.geomspace(100, 10000, 25)
sim = MultiSim(tracks, gen_car_lltd, rsd_range, k_c_r, "Roll Stiffness Distribution", "Chassis Roll Stiffness (lbf/deg)")

In [ ]:
sim.run_lltd()

In [ ]:
fig = go.Figure()
for i, k_c in enumerate(k_c_r):
    fig.add_trace(go.Scatter(x=rsd_range, y=sim.lltd[i, :], name=f"{k_c:.0f} lbf/deg"))

# make some vertical lines to show the roll stiffness distribution of the car
for rsd in [0.25, 0.64]:
    fig.add_trace(go.Scatter(x=[rsd, rsd], y=[0, 1], name=f"RSD {rsd:.2f}", line=dict(color="red")))
for rsd in [0.31, 0.50]:
    fig.add_trace(go.Scatter(x=[rsd, rsd], y=[0, 1], name=f"RSD {rsd:.2f}", line=dict(color="blue")))
# and some horizontal lines that are dashed to show the target LLTDs
for lltd in [0.25, 0.5]:
    fig.add_trace(go.Scatter(x=[0, 1], y=[lltd, lltd], name=f"LLTD {lltd:.2f}", line=dict(color="green")))

# set the rsd range of the plot to show
fig.update_xaxes(range=[0.2, 0.8])
# set the lltd range of the plot to show
fig.update_yaxes(range=[0.15, 0.8])
fig.update_layout(title="Lateral Load Transfer Distribution vs Roll Stiffness Distribution and Chassis Roll Stiffness", xaxis_title="Roll Stiffness Distribution", yaxis_title="Lateral Load Transfer Distribution", height=768, width=1024)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_contour(x=rsd_range, y=(k_c_r/(900 * FTLB_TO_NM)), z=np.abs(sim.lltd_diff) * 100, colorbar=dict(title="LLTD error %"), colorscale='Viridis', contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white',)),
            line_smoothing=0.85)
# fig.add_scatter(x=[0.2, 0.8], y=[1.5, 1.5], mode="lines", name="Target", line=dict(color="red", dash="dash"))
fig.update_layout(title="LLTD Error", xaxis_title="λ, Roll Stiffness Distribution (% Front)", yaxis_title="μ, Kc/(Kf+Kr)", height=768, width=1024)
# make the y axis log scale
fig.update_yaxes(type="log")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_contour(x=rsd_range, y=(k_c_r/(total_rsd * FTLB_TO_NM)), z=sim.lltd, colorbar=dict(title="LLTD (% Front)"), colorscale='Viridis', contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white',)),
            line_smoothing=0.85)
mu = (1400 * FTLB_TO_NM) / (total_rsd * FTLB_TO_NM)
fig.add_scatter(x=[0.2, 0.8], y=[mu, mu], mode="lines", name="Target", line=dict(color="red", dash="dash"))
fig.update_layout(title="LLTD Error", xaxis_title="λ, Roll Stiffness Distribution (% Front)", yaxis_title="μ, Kc/(Kf+Kr)", height=768, width=1024)
# make the y axis log scale
fig.update_yaxes(type="log")
fig.show()